In [1]:
import tensorflow as tf
from ASR_Network import ASR_Network
from DataPipe import DataPipeFactory
from util_function import inform_pooling

/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/p

In [2]:
# Disable all GPUS
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

In [3]:
ds = DataPipeFactory('../DataFolder/Tensorflow_DataRecord/Student_Answer_Record.tfrecord',
                     '../DataFolder/Siri_Related/Siri_Reference_Sample',
                     '../DataFolder/Siri_Related/Siri_Dense_Index', cache='../DataFolder/cache/datapipe/cached')
# dsp = ds.get_batch_data(10)
# it = iter(dsp)
ds.get_raw_data()

Load cache from ../DataFolder/cache/datapipe/cached


<_LoadDataset element_spec={'ref_mfcc': TensorSpec(shape=(4, None, 80), dtype=tf.float32, name=None), 'MatchSegment': TensorSpec(shape=(4, None), dtype=tf.int64, name=None), 'valid_ref_word': TensorSpec(shape=(4, None), dtype=tf.int64, name=None), 'MatchReference': TensorSpec(shape=(4, None), dtype=tf.int64, name=None), 'valid_ref_start': TensorSpec(shape=(4, None), dtype=tf.float32, name=None), 'RecordName': TensorSpec(shape=(), dtype=tf.string, name=None), 'passage_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'valid_ref_duration': TensorSpec(shape=(4, None), dtype=tf.float32, name=None), 'valid_stu_duration': TensorSpec(shape=(4, None), dtype=tf.float32, name=None), 'valid_stu_start': TensorSpec(shape=(4, None), dtype=tf.float32, name=None), 'stu_mfcc': TensorSpec(shape=(None, 80), dtype=tf.float32, name=None)}>

In [4]:
base_feature =([None,80], 512, [128,256,384,512], 3, 2)
dense_feature=([None, 1280], 512, [512,512], 2)
word_prediction = ([None, 512], 2500, [512,1024],2)
pooling_ratios = [125/(2**i) for i in range(4)]

In [5]:
C = ASR_Network(base_feature, dense_feature, word_prediction, pooling_ratios)

In [6]:
def unpack(d):
    value_s = d['stu_mfcc']
    start_s = tf.RaggedTensor.from_tensor(d['valid_stu_start'], padding=-1.)
    duration_s = tf.RaggedTensor.from_tensor(d['valid_stu_duration'], padding=-1.)

    #unpack with another key ref_mfcc, valid_ref_start, valid_ref_duration
    value_f = d['ref_mfcc']
    start_f = tf.RaggedTensor.from_tensor(d['valid_ref_start'], padding=-1.)
    duration_f = tf.RaggedTensor.from_tensor(d['valid_ref_duration'], padding=-1.)

    # unpack valid_ref_word

    words = tf.RaggedTensor.from_tensor(d['valid_ref_word'], padding=-1)
    return ((value_s, (start_s, duration_s)), (value_f, (start_f, duration_f))), words

In [7]:
dst = ds.get_batch_data(10, addition_map=unpack)

In [8]:
it = iter(dst)

2023-04-10 16:48:15.714367: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [9]:
data = next(it)

In [10]:
data

(((<tf.Tensor: shape=(10, 8687, 80), dtype=float32, numpy=
   array([[[-2.6056967e+00, -3.2185993e+00, -3.6952307e+00, ...,
            -2.8479064e+00, -3.1772044e+00, -3.1146815e+00],
           [-2.9376829e+00, -3.4104764e+00, -3.7282002e+00, ...,
            -2.8166041e+00, -3.0963197e+00, -2.9866042e+00],
           [-2.9196274e+00, -3.1949904e+00, -3.3803346e+00, ...,
            -2.6156731e+00, -2.8378875e+00, -2.8272674e+00],
           ...,
           [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, ...,
            -1.0000000e+00, -1.0000000e+00, -1.0000000e+00],
           [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, ...,
            -1.0000000e+00, -1.0000000e+00, -1.0000000e+00],
           [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, ...,
            -1.0000000e+00, -1.0000000e+00, -1.0000000e+00]],
   
          [[ 9.0534347e-01,  6.9793379e-01, -2.8725198e-01, ...,
            -1.9181389e+00, -1.6906520e+00, -2.0916131e+00],
           [ 1.0136919e+00,  5.1104045

In [11]:
C(data[0][0])

(<tf.RaggedTensor [[[-0.9112952, -0.5712936, 1.0788425, ..., -0.16803986, 0.038816832,
    -0.58681285],
   [0.03836789, -0.68311334, 2.8734608, ..., 0.8163837, -1.9241999,
    -2.7598693],
   [0.17634757, 0.22599325, 3.3377519, ..., 1.142943, -0.8193232,
    -2.6124535],
   ...,
   [-0.07157426, -0.69049656, 1.8811961, ..., 0.94472945, -1.1880833,
    -2.0477061],
   [-1.3137314, -1.8885258, 0.52405435, ..., 1.8025293, -0.19929887,
    -0.5584652],
   [-0.7343863, -1.6286553, 0.05009988, ..., 0.18325382, 1.0238619,
    -0.088704176]]                                                   ,
  [[0.08444274, -1.3813076, 2.172794, ..., -0.10712175, -0.8089412,
    -2.064732],
   [-0.71839195, -2.1740654, 1.2388828, ..., -0.3519473, -1.3112655,
    -2.4767683],
   [-0.9560158, -1.3056333, 0.47327936, ..., -0.17630738, -0.85512435,
    -0.6192228],
   ...,
   [0.06817873, -1.9652596, 0.21729991, ..., -0.24948326, 0.36578318,
    -0.4576192],
   [0.26244658, -2.4654875, 0.65144074, ..., -0.069155

In [12]:
C.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [13]:
C.fit(dst, epochs=1)

Instructions for updating:
Use fn_output_signature instead


Instructions for updating:
Use fn_output_signature instead


   8/1485 [..............................] - ETA: 3:54:21 - loss: 120.8901 - word_loss: 39.5584 - deep_loss: 81.3317

KeyboardInterrupt: 